In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14338,2022-01-06T17:00:00,ITA,19,Sicilia,38.115697,13.362357,944,119,1063,79053,...,DECEDUTI: N. 8 IL 05/01/22 - N. 6 IL 04/01/22 ...,12.0,NaN,Si informa che dalla data odierna per la defin...,406905.0,7823.0,3927151.0,4764640.0,ITG,ITG1
14339,2022-01-06T17:00:00,ITA,9,Toscana,43.769231,11.255889,903,98,1001,144612,...,NaN,12.0,NaN,NaN,393515.0,69508.0,5571893.0,3904910.0,ITI,ITI1
14340,2022-01-06T17:00:00,ITA,10,Umbria,43.106758,12.388247,181,12,193,31670,...,Si fa presente che 14 dei ricoveri non UTI app...,2.0,NaN,NaN,89374.0,16212.0,1375608.0,1627075.0,ITI,ITI2
14341,2022-01-06T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,42,5,47,4076,...,NaN,0.0,NaN,NaN,14169.0,4467.0,128139.0,245781.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14338,2022-01-06T17:00:00,19,Sicilia,944,119,1063,79053,80116,12918,14269,...,414728,8691791,3158013.0,12.0,406905.0,7823.0,3927151.0,4764640.0,ITG,ITG1
14339,2022-01-06T17:00:00,9,Toscana,903,98,1001,144612,145613,12481,17286,...,463023,9476803,3772229.0,12.0,393515.0,69508.0,5571893.0,3904910.0,ITI,ITI1
14340,2022-01-06T17:00:00,10,Umbria,181,12,193,31670,31863,1279,3206,...,105586,3002683,557715.0,2.0,89374.0,16212.0,1375608.0,1627075.0,ITI,ITI2
14341,2022-01-06T17:00:00,2,Valle d'Aosta,42,5,47,4076,4123,470,551,...,18636,373920,114933.0,0.0,14169.0,4467.0,128139.0,245781.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-06', '2022-01-05')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-06T17:00:00,1,Piemonte,1513,136,1649,119115,120764,9830,14103,...,560914,11722925,3178330.0,21.0,407884.0,153030.0,4221974.0,7500951.0,ITC,ITC1
1,2022-01-06T17:00:00,2,Valle d'Aosta,42,5,47,4076,4123,470,551,...,18636,373920,114933.0,0.0,14169.0,4467.0,128139.0,245781.0,ITC,ITC2
2,2022-01-06T17:00:00,3,Lombardia,2519,229,2748,428821,431569,40124,52693,...,1432864,25934786,7003398.0,25.0,1044951.0,387913.0,13304002.0,12630784.0,ITC,ITC4
3,2022-01-06T17:00:00,5,Veneto,1167,187,1354,149860,151214,14491,18129,...,721879,20225774,2497733.0,21.0,609509.0,112370.0,8435757.0,11790017.0,ITH,ITH3
4,2022-01-06T17:00:00,6,Friuli Venezia Giulia,306,30,336,26046,26382,3350,4159,...,171927,4643373,980294.0,5.0,143217.0,28710.0,2693146.0,1950227.0,ITH,ITH4
5,2022-01-06T17:00:00,7,Liguria,603,44,647,14798,15445,398,3066,...,160937,3641239,1064293.0,5.0,160937.0,0.0,2036206.0,1605033.0,ITC,ITC3
6,2022-01-06T17:00:00,8,Emilia-Romagna,1744,140,1884,154904,156788,34967,38528,...,627222,11953763,2429146.0,18.0,578977.0,48245.0,7435254.0,4518509.0,ITH,ITH5
7,2022-01-06T17:00:00,9,Toscana,903,98,1001,144612,145613,12481,17286,...,463023,9476803,3772229.0,12.0,393515.0,69508.0,5571893.0,3904910.0,ITI,ITI1
8,2022-01-06T17:00:00,10,Umbria,181,12,193,31670,31863,1279,3206,...,105586,3002683,557715.0,2.0,89374.0,16212.0,1375608.0,1627075.0,ITI,ITI2
9,2022-01-06T17:00:00,11,Marche,243,50,293,9855,10148,1120,3120,...,155169,2086351,1236351.0,1.0,155169.0,0.0,1682388.0,403963.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-05T17:00:00,1,Piemonte,1460,122,1582,109352,110934,12074,16937,...,546811,11650822,3159891.0,16.0,408405.0,138406.0,4208038.0,7442784.0,ITC,ITC1
1,2022-01-05T17:00:00,2,Valle d'Aosta,45,5,50,3603,3653,564,630,...,18085,370628,114082.0,1.0,14057.0,4028.0,127825.0,242803.0,ITC,ITC2
2,2022-01-05T17:00:00,3,Lombardia,2420,226,2646,388799,391445,47483,51587,...,1380171,25704386,6963167.0,8.0,1026497.0,353674.0,13232726.0,12471660.0,ITC,ITC4
3,2022-01-05T17:00:00,5,Veneto,1233,196,1429,135294,136723,12364,16871,...,703750,20088236,2490171.0,18.0,604156.0,99594.0,8404636.0,11683600.0,ITH,ITH3
4,2022-01-05T17:00:00,6,Friuli Venezia Giulia,309,29,338,22694,23032,2954,3931,...,167777,4616036,976655.0,6.0,141381.0,26396.0,2683015.0,1933021.0,ITH,ITH4
5,2022-01-05T17:00:00,7,Liguria,562,45,607,14440,15047,597,2231,...,157871,3616544,1060735.0,3.0,157871.0,0.0,2025691.0,1590853.0,ITC,ITC3
6,2022-01-05T17:00:00,8,Emilia-Romagna,1622,141,1763,120058,121821,11063,13671,...,588695,11891291,2422903.0,15.0,567815.0,20880.0,7402413.0,4488878.0,ITH,ITH5
7,2022-01-05T17:00:00,9,Toscana,871,90,961,132171,133132,14100,16957,...,445737,9401279,3749298.0,11.0,387268.0,58469.0,5549004.0,3852275.0,ITI,ITI1
8,2022-01-05T17:00:00,10,Umbria,185,11,196,30388,30584,2768,3967,...,102380,2981367,552678.0,1.0,88211.0,14169.0,1370226.0,1611141.0,ITI,ITI2
9,2022-01-05T17:00:00,11,Marche,244,52,296,8732,9028,362,2147,...,152049,2067064,1221478.0,3.0,152049.0,0.0,1672575.0,394489.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-06T17:00:00,1,Piemonte,1513,136,1649,119115,120764,9830,14103,...,11722925,3178330.0,21.0,407884.0,153030.0,4221974.0,7500951.0,ITC,ITC1,4.341375e+06
1,2022-01-06T17:00:00,2,Valle d'Aosta,42,5,47,4076,4123,470,551,...,373920,114933.0,0.0,14169.0,4467.0,128139.0,245781.0,ITC,ITC2,1.255010e+05
2,2022-01-06T17:00:00,3,Lombardia,2519,229,2748,428821,431569,40124,52693,...,25934786,7003398.0,25.0,1044951.0,387913.0,13304002.0,12630784.0,ITC,ITC4,1.010397e+07
3,2022-01-06T17:00:00,5,Veneto,1167,187,1354,149860,151214,14491,18129,...,20225774,2497733.0,21.0,609509.0,112370.0,8435757.0,11790017.0,ITH,ITH3,4.907704e+06
4,2022-01-06T17:00:00,6,Friuli Venezia Giulia,306,30,336,26046,26382,3350,4159,...,4643373,980294.0,5.0,143217.0,28710.0,2693146.0,1950227.0,ITH,ITH4,1.211357e+06
5,2022-01-06T17:00:00,7,Liguria,603,44,647,14798,15445,398,3066,...,3641239,1064293.0,5.0,160937.0,0.0,2036206.0,1605033.0,ITC,ITC3,1.543127e+06
6,2022-01-06T17:00:00,8,Emilia-Romagna,1744,140,1884,154904,156788,34967,38528,...,11953763,2429146.0,18.0,578977.0,48245.0,7435254.0,4518509.0,ITH,ITH5,4.467118e+06
7,2022-01-06T17:00:00,9,Toscana,903,98,1001,144612,145613,12481,17286,...,9476803,3772229.0,12.0,393515.0,69508.0,5571893.0,3904910.0,ITI,ITI1,3.722729e+06
8,2022-01-06T17:00:00,10,Umbria,181,12,193,31670,31863,1279,3206,...,3002683,557715.0,2.0,89374.0,16212.0,1375608.0,1627075.0,ITI,ITI2,8.802850e+05
9,2022-01-06T17:00:00,11,Marche,243,50,293,9855,10148,1120,3120,...,2086351,1236351.0,1.0,155169.0,0.0,1682388.0,403963.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-06T17:00:00,1,Piemonte,1513,136,1649,119115,120764,9830,14103,...,7500951.0,ITC,ITC1,4.341375e+06,Piemonte,1460,5824,122,628,99
1,2022-01-06T17:00:00,2,Valle d'Aosta,42,5,47,4076,4123,470,551,...,245781.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,45,99,5,33,0
2,2022-01-06T17:00:00,3,Lombardia,2519,229,2748,428821,431569,40124,52693,...,12630784.0,ITC,ITC4,1.010397e+07,Lombardia,2420,10457,226,1530,0
3,2022-01-06T17:00:00,5,Veneto,1167,187,1354,149860,151214,14491,18129,...,11790017.0,ITH,ITH3,4.907704e+06,Veneto,1233,6000,196,1000,0
4,2022-01-06T17:00:00,6,Friuli Venezia Giulia,306,30,336,26046,26382,3350,4159,...,1950227.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,309,1277,29,175,0
5,2022-01-06T17:00:00,7,Liguria,603,44,647,14798,15445,398,3066,...,1605033.0,ITC,ITC3,1.543127e+06,Liguria,562,1762,45,219,12
6,2022-01-06T17:00:00,8,Emilia-Romagna,1744,140,1884,154904,156788,34967,38528,...,4518509.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1622,9001,141,889,0
7,2022-01-06T17:00:00,9,Toscana,903,98,1001,144612,145613,12481,17286,...,3904910.0,ITI,ITI1,3.722729e+06,Toscana,871,5033,90,570,24
8,2022-01-06T17:00:00,10,Umbria,181,12,193,31670,31863,1279,3206,...,1627075.0,ITI,ITI2,8.802850e+05,Umbria,185,662,11,86,41
9,2022-01-06T17:00:00,11,Marche,243,50,293,9855,10148,1120,3120,...,403963.0,ITI,ITI3,1.518400e+06,Marche,244,1007,52,252,53


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-06T17:00:00,3,Lombardia,2519,229,2748,428821,431569,40124,52693,...,14.97,40022,1106,230400,40231.0,46,12523,22.87,0.52151,14.18
1,2022-01-06T17:00:00,8,Emilia-Romagna,1744,140,1884,154904,156788,34967,38528,...,15.75,34846,24857,62472,6243.0,24,3536,61.67,0.86248,14.04
2,2022-01-06T17:00:00,5,Veneto,1167,187,1354,149860,151214,14491,18129,...,18.70,14566,1258,137538,7562.0,14,3624,13.18,0.36940,14.71
3,2022-01-06T17:00:00,9,Toscana,903,98,1001,144612,145613,12481,17286,...,17.19,12441,329,75524,22931.0,18,4787,22.89,0.46434,12.44
4,2022-01-06T17:00:00,15,Campania,868,73,941,134153,135094,12531,16512,...,10.61,12468,-460,116411,22181.0,7,3974,14.18,0.28539,11.31
5,2022-01-06T17:00:00,19,Sicilia,944,119,1063,79053,80116,12918,14269,...,13.98,12883,6941,59933,58310.0,19,1332,23.81,0.28719,8.35
6,2022-01-06T17:00:00,1,Piemonte,1513,136,1649,119115,120764,9830,14103,...,21.66,9763,-2834,72103,18439.0,12,4261,19.56,0.32485,12.92
7,2022-01-06T17:00:00,12,Lazio,1371,184,1555,128758,130313,12450,14055,...,19.51,12372,-2409,111745,4440.0,7,1598,12.58,0.23962,9.74
8,2022-01-06T17:00:00,16,Puglia,384,38,422,45226,45648,4486,5558,...,7.71,4466,44,75641,11382.0,8,1064,7.35,0.13866,8.33
9,2022-01-06T17:00:00,13,Abruzzo,248,24,272,37705,37977,4662,4808,...,13.26,4640,-507,37405,5486.0,1,144,12.85,0.36821,9.87


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,25,65,90,27.78
1,Piemonte,136,492,628,21.66
2,Liguria,44,175,219,20.09
3,Marche,50,202,252,19.84
4,Lazio,184,759,943,19.51
5,Veneto,187,813,1000,18.70
6,Toscana,98,472,570,17.19
7,Friuli Venezia Giulia,30,145,175,17.14
8,P.A. Bolzano,17,83,100,17.00
9,Calabria,30,159,189,15.87


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))